In [ ]:
from pathlib import Path
import re
import csv
import datetime
import time
import io

import requests
from bs4 import BeautifulSoup

from stock.util import convert_to_number

In [ ]:
def get_stock_data(code: str, base_url: str = "https://kabutan.jp/stock/kabuka?code={}&ashi=day"):
    res = requests.get(base_url.format(code))
    soup = BeautifulSoup(res.text)

    daily_data = []
    stock_tables = soup.find("div", {"id": "stock_kabuka_table"})
    if stock_tables is None:
        return daily_data
    for table in stock_tables.find_all("table"):
        tbody = table.find("tbody")
        if tbody is None:
            continue
        for table_row in tbody.find_all("tr"):
            thead = table_row.find("th")
            tdata = table_row.find_all("td")
            if thead is None or len(tdata) < 7:
                break
            date = datetime.datetime.strptime(thead.text, "%y/%m/%d")
            start = convert_to_number(tdata[0].text)
            high = convert_to_number(tdata[1].text)
            low = convert_to_number(tdata[2].text)
            end = convert_to_number(tdata[3].text)
            volume = convert_to_number(tdata[6].text)
            daily_data.append([date, start, high, low, end, volume])
    return daily_data

In [ ]:
def update_csv(csv_path: Path, data: list):
